In [ ]:
import matrixmethod
import matrixmethod_cython
import numpy as np
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
from Anton_XRR import reflectivity
import pint
u = pint.UnitRegistry()
%load_ext Cython

In [ ]:
n = np.array([1, 1-1e-5+1e-6j, 1-2e-5+2e-6j])
thick = np.array([100.])
wl = 0.15
ang_deg = np.linspace(0, 1, 1001)[1:]
ang = np.deg2rad(ang_deg)

%timeit matrixmethod.reflec_and_trans(n, wl, ang, thick)
r, t = matrixmethod.reflec_and_trans(n, wl, ang, thick)
r = np.array(r)
t = np.array(t)

%timeit matrixmethod_cython.reflec_and_trans(n, wl, ang, thick)
rrr, ttt = matrixmethod_cython.reflec_and_trans(n, wl, ang, thick)
rrr = np.array(rrr)
ttt = np.array(ttt)

def anton_xrr(ang_deg, wl, n):
    l = len(ang_deg)
    AOI = 90.-ang_deg
    wavelengths = np.array([wl]*l)
    sigma = np.array([0, 0])
    t = np.array(thick + [0.])
    t = np.outer(t, np.ones(l))
    n = np.outer(n, np.ones(l))
    kz_AOI, kx_AOI = reflectivity.helper.k_z_generator(AOI, wavelengths, n)
    r, trans = reflectivity.matrixmethod_xrr.amplitudes(n, wavelengths, kz_AOI, t, sigma)
    return r, trans
%timeit anton_xrr(ang_deg, wl, n)
rr, tt = anton_xrr(ang_deg, wl, n)

In [ ]:
plt.figure()
plt.plot(ang_deg, np.abs(r)**2)
plt.plot(ang_deg, np.abs(rr[0])**2)
plt.plot(ang_deg, np.abs(rrr)**2)
#plt.plot(ang_deg, np.abs(t)**2)
plt.yscale('log')